In [2]:
embeding_matrix = load_pickle_file("./embeding_matrix")
word2index = load_pickle_file("./word2index")
index_sequences = load_pickle_file("./index_sequences.pkl")
labels = load_pickle_file("./labels")
SEQUENCE_LENGTH = 618
VOCAB_SIZE = len(embeding_matrix)
EMB_DIMENSION = 300
VALIDATION_SPLIT = 0.1
PAD_SEQ_VALUE = 3001066


In [3]:
import numpy as np
same_lenght_seq = [seq[:SEQUENCE_LENGTH].reshape(1,SEQUENCE_LENGTH) if len(seq) >= SEQUENCE_LENGTH 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH) 
                   for seq in index_sequences]
same_lenght_seq = np.concatenate(same_lenght_seq,axis=0)


In [4]:
import random
test_size = int(len(labels) * 0.2)
true_labels = np.where(np.array(labels)==0)[0]
start_index = true_labels[0]
random_test_true_indexes = [int(x * len(true_labels)+start_index) for x in np.random.random(test_size//2)]
random_test_false_indexes = [int(x * (len(labels) - len(true_labels))) for x in np.random.random(test_size//2)]
x_test_indexes = random_test_true_indexes + random_test_false_indexes
random.shuffle(x_test_indexes)

x_train_indexes = [x for x in range(0,len(labels)) if x not in x_test_indexes]

random.shuffle(x_train_indexes)

labels = np.vstack(np.array([1,0]).reshape(1,2) if x == 0 else np.array([0,1]).reshape(1,2) for x in labels)

y_test = np.array(labels)[x_test_indexes]
y_train = np.array(labels)[x_train_indexes]

x_train = same_lenght_seq[x_train_indexes]
x_test = same_lenght_seq[x_test_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


Dataset split

In [4]:
# from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(same_lenght_seq, labels, test_size=0.2, random_state=42,shuffle = True)

In [5]:
data_size = len(x_train)
perm = np.random.permutation(data_size)
idx_train = perm[:int(data_size*(1-VALIDATION_SPLIT))]
idx_val = perm[int(data_size*(1-VALIDATION_SPLIT)):]

data_train = x_train[idx_train]
labels_train = y_train[idx_train]

data_val = x_train[idx_val]
labels_val = y_train[idx_val]

In [13]:
data_train.shape

(7380, 434)

<h1>Convolutional neural networks</h1>

In [6]:
import tensorflow as tf
from tensorflow import keras as tk
from tensorflow.keras import layers as tfkl
from sklearn.metrics import mean_squared_error,mean_absolute_error

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Conv1D, MaxPool1D, SpatialDropout1D, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.models import Model
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [7]:
model_checkpoint = ModelCheckpoint("./Checkpoints", save_best_only=True, save_weights_only=True)


In [7]:
NUM_FILTERS = 20
KERNEL_SIZE = 18
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15
conv_model = tk.Sequential()
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH,
        trainable=False)
conv_model.add(embedding_layer)
# conv_model.add(SpatialDropout1D())
conv_model.add(Conv1D(NUM_FILTERS, KERNEL_SIZE, activation = 'relu',kernel_regularizer =tf.keras.regularizers.l2(l=0.01)))
conv_model.add(Dropout(0.2))
# conv_model.add(BatchNormalization())
conv_model.add(MaxPool1D(pool_size = 12))
conv_model.add(GlobalMaxPool1D())
# conv_model.add(Flatten())
conv_model.add(Dense(2,activation='sigmoid'))


In [8]:
conv_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 618, 300)          900320100 
_________________________________________________________________
conv1d (Conv1D)              (None, 601, 20)           108020    
_________________________________________________________________
dropout (Dropout)            (None, 601, 20)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 50, 20)            0         
_________________________________________________________________
global_max_pooling1d (Global (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 42        
Total params: 900,428,162
Trainable params: 108,062
Non-trainable params: 900,320,100
____________________________________

In [21]:
del conv_model

In [23]:
conv_model.reset_states()

In [9]:
opt = tk.optimizers.Adam(learning_rate=0.01)

In [10]:
conv_model.compile(loss='binary_crossentropy',
        optimizer=opt,
        metrics=['acc'])
history2 = conv_model.fit(data_train,labels_train,epochs = 10,batch_size = 256, validation_data = [data_val,labels_val],shuffle=True)

Train on 11718 samples, validate on 1302 samples
Epoch 1/10
11718/11718 [==============================] - 48s 4ms/sample - loss: 0.5756 - acc: 0.7971 - val_loss: 0.5264 - val_acc: 0.8349
Epoch 2/10
11718/11718 [==============================] - 47s 4ms/sample - loss: 0.4734 - acc: 0.8722 - val_loss: 0.4880 - val_acc: 0.8952
Epoch 3/10
11718/11718 [==============================] - 47s 4ms/sample - loss: 0.4367 - acc: 0.9078 - val_loss: 0.4638 - val_acc: 0.9132
Epoch 4/10
11718/11718 [==============================] - 46s 4ms/sample - loss: 0.4043 - acc: 0.9222 - val_loss: 0.4468 - val_acc: 0.9075
Epoch 5/10
11718/11718 [==============================] - 48s 4ms/sample - loss: 0.4192 - acc: 0.9213 - val_loss: 0.4303 - val_acc: 0.9209
Epoch 6/10
11718/11718 [==============================] - 47s 4ms/sample - loss: 0.3961 - acc: 0.9278 - val_loss: 0.4302 - val_acc: 0.9255
Epoch 7/10
11718/11718 [==============================] - 49s 4ms/sample - loss: 0.4020 - acc: 0.9300 - val_loss: 0.4

In [14]:
history2 = conv_model.fit(data_train,labels_train,epochs = 10,batch_size = 256, validation_data = [data_val,labels_val],shuffle=True)

Train on 11718 samples, validate on 1302 samples
Epoch 1/10
11718/11718 [==============================] - 48s 4ms/sample - loss: 0.3972 - acc: 0.9434 - val_loss: 0.4076 - val_acc: 0.9485
Epoch 2/10
11718/11718 [==============================] - 47s 4ms/sample - loss: 0.3949 - acc: 0.9440 - val_loss: 0.4153 - val_acc: 0.9409
Epoch 3/10
11718/11718 [==============================] - 49s 4ms/sample - loss: 0.4063 - acc: 0.9403 - val_loss: 0.4336 - val_acc: 0.9424
Epoch 4/10
11718/11718 [==============================] - 47s 4ms/sample - loss: 0.4117 - acc: 0.9438 - val_loss: 0.4288 - val_acc: 0.9389
Epoch 5/10
11718/11718 [==============================] - 35s 3ms/sample - loss: 0.4083 - acc: 0.9423 - val_loss: 0.4735 - val_acc: 0.9401
Epoch 6/10
11718/11718 [==============================] - 35s 3ms/sample - loss: 0.4214 - acc: 0.9439 - val_loss: 0.4278 - val_acc: 0.9462
Epoch 7/10
11718/11718 [==============================] - 37s 3ms/sample - loss: 0.3993 - acc: 0.9444 - val_loss: 0.3

In [9]:
import os
print(os.getcwd())

/home/gorgiev_laze/NLP/Modeling-dataset-2


In [21]:
conv_model.save("./Checkpoints/conv_model2.h5")

In [22]:
conv_model.save_weights("./Checkpoints/conv_weights2")

In [7]:
conv_model = tf.keras.models.load_model("./Checkpoints/conv_model.h5")

In [10]:
conv_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 434, 300)          900320100 
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 423, 12)           43212     
_________________________________________________________________
dropout_3 (Dropout)          (None, 423, 12)           0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 28, 12)            0         
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 12)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 26        
Total params: 900,363,338
Trainable params: 43,238
Non-trainable params: 900,320,100
___________________________________

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

pred = conv_model.predict(x_test)
pred = [np.argmax(x) for x in pred]

print(f"F1-score : {f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)}")
print(f"Accuracy-score : {accuracy_score([np.argmax(x) for x in y_test],pred)}")

print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")

F1-score : 0.9414957780458384
Accuracy-score : 0.9387239418825016
Precision : 0.9007501442585113
Recall  : 0.9861023373341756


In [17]:
from sklearn.metrics import roc_auc_score
roc_auc_score([np.argmax(x) for x in y_test],pred)

0.9387239418825015

In [29]:
from sklearn.metrics import accuracy_score

pred = conv_model.predict(x_test)
pred = [np.argmax(x) for x in pred]



In [30]:
from sklearn.metrics import f1_score
f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)

0.9362930077691454

In [31]:
accuracy_score([np.argmax(x) for x in y_test],pred)

0.9093779602147143

In [32]:
from sklearn.metrics import roc_auc_score
roc_auc_score([np.argmax(x) for x in y_test],pred)

0.8797661587489404

In [34]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 0)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 0)}")

Precision : 0.8851894374282434
Recall  : 0.8048016701461378


In [1]:
import pickle
def save_pickle_file(file,file_path,protocol=None):
    with open(file_path,"wb") as f:
        if protocol:
            pickle.dump(file,f,protocol = protocol)
        else:
            pickle.dump(file,f)

def load_pickle_file(file_path, protocol=None):
     with open(file_path,"rb") as f:
        if protocol:
            return pickle.load(f)
        else:
            return pickle.load(f)
